In [1]:
import warnings
warnings.filterwarnings('ignore')

## Diagnosing the model

The diagnosis step checks if the MNLE has learned the likelihood *P(observations | parameters, experimental conditions)* of its corresponding DDM. If the likelihood is well approximated, the MNLE emulates the behaviour of the simulator. To check the training performance the MNLE is evaluated on the simulated test data using four different diagnostics.   
Since diagnosis can take quite some time, a fast diagnosis and a slower diagnosis can be run separately. The fast diagnosis allows to observe the behaviour of the MNLE in order to decide if the training should be run again, possibly while increasing `n_wandb_sweeps` in `config/algorithm` or increasing the number of training data simulations in *config/task*. If the results of the fast diagnosis look satisfactory, the slow diagnosis determines if the posterior distribution *P(parameters | observations, experimental conditions)* is well defined.

#### Fast diagnosis

The fast diagnosis utilizes the test data containing multiple i.i.d. observations per combination of inputs (i.e. parameters and experimental conditions). The number of plots that will be created per diagnosis test equals `sim_iid_test_data_params.num_params`.  

**Compare observations**

This test compares observations simulated by the DDM simulator to observations sampled from the MNLE using the same input parameters and experimental conditions. A well trained MNLE should return observations similar to the simulated observations.  
The following figure shows one example plot. The caption of the plot describes the input used for simulating and sampling the observations. The pink histogram shows simulated observations while the blue histogram corresponds to observations sampled from the MNLE.  
You will find the compare observation plots at *diagnose/compare_observations.png* in your result folder.

<img src="tutorial_images/compare_observations.png" width=1000px>


**[Posterior Predictive Check](https://rss.onlinelibrary.wiley.com/doi/full/10.1111/rssa.12378)**

The posterior predictive check allows you to investigate the posterior computed by the MNLE.  
Take a look at the example plot below. On the left you can see a blue histogram describing the posterior approximation *P(parameters | observations, experimental conditions)* of each parameter given the simulated test observations plotted on the right. The pink lines describe the ground truth parameters used for simulating the test observations. If the posterior is well approximated, the pink lines should lie somewhere within the blue histogram, as shown below. The grey outline shows an approximation of the prior. If a parameter has been learned correctly, its posterior shape should usually differ from its prior shape, especially if the prior is uniform.  
The histogram of simulated samples on the right shows observations that have been generated using the posterior samples. If the posterior samples have been inferred correctly from the test observations, they should allow to simulate observations that are close to the test observations. Therefore, both pink histograms should look similar.

For more information about the posterior predictive check read [sbi tutorial 12](https://github.com/mackelab/sbi/blob/main/tutorials/12_diagnostics_posterior_predictive_check.ipynb).

<img src="tutorial_images/posterior_predictive.png" width=1000px>

#### Slow diagnosis

As mentioned before, the slow diagnosis checks if the posterior is well defined. The top left plot in the subsequent image shows you an example ground truth posterior in green. Posterior approximations based on the trained MNLE might be shifted to the left or right, this is called positive bias or negative bias. Moreover, their standard deviation might be too large (overdispersion) or too narrow (underdispersion).

**[Simulation-Based Calibration (SBC)](https://arxiv.org/abs/1804.06788)**

SBC checks if a bias and/or over-/underdispersion has been found in the posterior approximation. The bottom left image visualizes the test results. A distribution is assumed to be well defined if its line lies within the grey area that represents the 99% confidence region. You can see that this applies to the ground truth plotted in green. The remaining lines serve as a reference for the discernible pattern that each of the poorly defined posteriors will produce.  

The plot on the right shows an example of the image the pipeline will create and save in *diagnose/sbc.png* in your result folder. You can observe one line for each parameter. 

Read [sbi tutorial 13](https://github.com/mackelab/sbi/blob/main/tutorials/13_diagnostics_simulation_based_calibration.ipynb) for more information about SBC.

<img src="tutorial_images/sbc.PNG" width=1100px>


**[Local coverage test](https://proceedings.mlr.press/v161/zhao21b.html)**

The local coverage test is similar to SBC, but tests the posterior distribution locally instead of globally. 

An overview of the local coverage is provided by utilizing the local results to compute a global p-value for each parameter. The p-values are saved in *diagnose/global_pvalues.json* as visualized in the figure below. A p-value < 0.05 indicates that the posterior is not well defined con respect to this parameter.

<img src="tutorial_images/pvalues.PNG">

The plots saved in *diagnose/local_coverage_check.png* will allow you to get a closer look at the local coverage performance. The subsequent example image visualizes local coverage tests on the posterior *P(parameter | observations, experimental conditions)* specified in the subplot title.  
*diagnose/local_coverage_check.png* only contains plots showing local coverage tests that indicated a bias and/or over-/underdispersion. In the example plot below a slight positive bias pattern for the drift parameter is identifiable. The drift bias is in accordance with the small global p-value that has been computed for the drift parameter. If all local coverage tests of a parameter indicated the posterior being well defined, no plots are drawn for this parameter. You can choose the maximum number of plots drawn per parameter by adapting `n_local_coverage_plots` in *config/tasks*.

Read [sbi tutorial 14]() for more information about the local coverage test.

<img src="tutorial_images/local_coverage.png" width=450px>

## Improve results

The easiest way to improve the model performance might lie in incrementing the training data by means of increasing the number of training simulations in *config/task* and repeating the simulation step. If the simulator is slow, see **tutorial_7** to reuse the previously simulated data and add simulations to it.

The WandB visualization allows to monitor the training performance. The blue boxes in **tutorial_3** describe how to interpret the visualization and adapt the search ranges of hyperparameters towards the best configuration. Additionally, the number of training runs can be increased via `n_wandb_sweeps` in *config/algorithm*.

The posterior result can be improved by increasing thinning, the number of chains or the number of warmup steps. Changing the initial strategy from `proposal` to `sir` might improve the initial locations of chains since not only the prior probability but also the likelihood is taken into account. When using `sir`, the number of initial candidate locations can be increased via `init_strategy_num_candidates`.  
Selecting the [No-U-Turn sampler](https://www.jmlr.org/papers/volume15/hoffman14a/hoffman14a.pdf) might yield better posteriors than Hamiltonian Monte Carlo due to needing less tuning. See **tutorial_4** for a description of the posterior parameters.


## Run fast diagnosis step




In [2]:
dir = '../results/${result_folder}'

In [6]:
%run ../ddm_stride/run.py hydra.run.dir={dir} run_diagnose_fast=True

no viable alternative at input '[{'name''
See https://hydra.cc/docs/next/advanced/override_grammar/basic for details


Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.


## Run slow diagnosis step



In [ ]:
%run ../ddm_stride/run.py hydra.run.dir={dir} run_diagnose_slow=True

no viable alternative at input '[{'name''
See https://hydra.cc/docs/next/advanced/override_grammar/basic for details


Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.
